In [3]:
import numpy as np
import cv2
import torch
import torchvision
import matplotlib.pyplot as plt

In [24]:
img_size = 400
sub_sample = 8

In [25]:
dummy_img = torch.zeros((1, 3, img_size, img_size)).float()
print(dummy_img.shape)

torch.Size([1, 3, 400, 400])


In [26]:
model = torchvision.models.vgg16(pretrained=True)
fe = list(model.features)

In [27]:
req_features = []
k = dummy_img.clone()
for i in fe:
    k = i(k)
    if k.size()[2] < img_size//sub_sample:
        break
    req_features.append(i)
    out_channels = k.size()[1]
print(len(req_features)) #30
print(out_channels) # 512

23
512


In [28]:
ratios = [0.5, 1, 2]
scales = [sub_sample//2, sub_sample, sub_sample*2]

anchor_base = np.zeros((len(ratios) * len(scales), 4), dtype=np.float32)

In [29]:
ctr_y = sub_sample / 2.
ctr_x = sub_sample / 2.

print(ctr_y, ctr_x)
# Out: (8, 8)
for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
        h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
        w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])

        index = i * len(anchor_scales) + j

        anchor_base[index, 0] = ctr_y - h / 2.
        anchor_base[index, 1] = ctr_x - w / 2.
        anchor_base[index, 2] = ctr_y + h / 2.
        anchor_base[index, 3] = ctr_x + w / 2.

4.0 4.0


In [35]:
fe_size = (img_size//sub_sample)
ctr_x = np.arange(sub_sample, (fe_size+1) * sub_sample, sub_sample)
ctr_y = np.arange(sub_sample, (fe_size+1) * sub_sample, sub_sample)

In [57]:
ctr = np.transpose(np.meshgrid(ctr_y.flatten(), ctr_x.flatten()), (1,2,0)).reshape(-1,2)-sub_sample//2

In [88]:
anchors = np.zeros((fe_size * fe_size * 9, 4))
index = 0
for c in ctr:
    ctr_y, ctr_x = c
    for i in range(len(ratios)):
        for j in range(len(anchor_scales)):
            h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
            w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
            anchors[index, 0] = ctr_y - h / 2.
            anchors[index, 1] = ctr_x - w / 2.
            anchors[index, 2] = ctr_y + h / 2.
            anchors[index, 3] = ctr_x + w / 2.
            index += 1

In [91]:
index_inside = np.where(
        (anchors[:, 0] >= 0) &
        (anchors[:, 1] >= 0) &
        (anchors[:, 2] < 400) &
        (anchors[:, 3] < 400)
    )[0]
print(index_inside.shape)

(14732,)


In [93]:
label = np.ones(index_inside.shape,dtype='int32')*-1
valid_anchor_boxes = anchors[index_inside]

In [96]:
valid_anchor_boxes

array([[  5.372583 ,   0.6862915,  50.627417 ,  23.3137085],
       [ 13.372583 ,   0.6862915,  58.627417 ,  23.3137085],
       [ 21.372583 ,   0.6862915,  66.627417 ,  23.3137085],
       ...,
       [333.372583 , 376.6862915, 378.627417 , 399.3137085],
       [341.372583 , 376.6862915, 386.627417 , 399.3137085],
       [349.372583 , 376.6862915, 394.627417 , 399.3137085]])